In [15]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent, LlmAgent, BaseAgent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search
from google.genai import types # For creating message Content/Parts
import warnings
from dotenv import load_dotenv

load_dotenv()
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("Libraries imported.")

Libraries imported.


In [16]:
from python.tools.craw4ai import crawl
from python.tools.webscraper import web_grounding

In [17]:
search_agent = Agent(
    name="search_agent",
    model= 'gemini-2.0-flash',
    instruction=    "Gather the most relevant information from the web and summarize it. "
                    "You can use the following tools to help you with this task: "
                    "- Use the `web_grounding(query:str)` tool to search anything on the web."
                    "- Use the `crawl(url:str)` tool to crawl a website and extract information from it."
                    "You need to crawl the website to get the information you need."
                    "Not only from web_grounding."
                    "You can use each tool multiple times."
                    "You can also use the tools in any order you want."
    ,
    description="A search agent that uses tools to find analyse and summarize information.",
    tools=[web_grounding, crawl],
)

In [18]:
session_service = InMemorySessionService()
APP_NAME = "test_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=search_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='test_app', User='user_1', Session='session_001'
Runner created for agent 'search_agent'.


In [19]:
def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.get_function_calls:
            calls = event.get_function_calls()
            if calls:
                for call in calls:
                    tool_name = call.name
                    arguments = call.args # This is usually a dictionary
                    print(f"  Tool: {tool_name}, Args: {arguments}")
                    # Application might dispatch execution based on this
    
        if event.is_final_response():
            final_response = event.content.parts[0].text
            # print(final_response)
            return final_response

In [20]:
query = "Pick one good financial health with cheap price stock to invest in thailand now."
r = call_agent(query)

  Tool: web_grounding, Args: {'query': 'best value stocks Thailand stock exchange'}
  Tool: crawl, Args: {'url': 'https://www.bangkokpost.com/business/investment/2970028/kcs-picks-12-promising-thai-stocks-amid-slump'}
[INIT].... → Crawl4AI 0.6.2
[FETCH]... ↓ https://www.bangkokpost.com/business/investment/...28/kcs-picks-12-promising-thai-stocks-amid-slump  | ✓ | ⏱: 2.52s
[SCRAPE].. ◆ https://www.bangkokpost.com/business/investment/...28/kcs-picks-12-promising-thai-stocks-amid-slump  | ✓ | ⏱: 0.17s
[COMPLETE] ● https://www.bangkokpost.com/business/investment/...28/kcs-picks-12-promising-thai-stocks-amid-slump  | ✓ | ⏱: 2.71s
  Tool: crawl, Args: {'url': 'https://www.nationthailand.com/blogs/business/trading-investment/40044586'}
[INIT].... → Crawl4AI 0.6.2
[FETCH]... ↓ https://www.nationthailand.com/blogs/business/trading-investment/40044586                            | ✓ | ⏱: 5.20s
[SCRAPE].. ◆ https://www.nationthailand.com/blogs/business/trading-investment/40044586                  

In [21]:
print("Response from agent:")
print(r)

Response from agent:
The Nation Thailand article lists the 10 SET100 stocks that provided the highest dividend yields in 2024:

1.  Jasmine International Pcl (JAS)
2.  SCB X Pcl (SCB)
3.  Tipco Asphalt Pcl (TASCO)
4.  Thanachart Capital Pcl (TCAP)
5.  Kiatnakin Phatra Bank Pcl (KKP)
6.  Sri Trang Agro-Industry Pcl (STA)
7.  TMBThanachart Bank Pcl (TTB)
8.  Sri Trang Gloves (Thailand) Pcl (STGT)
9.  Thai Life Insurance Pcl (TLI)
10. Bangkok Airways Pcl (BA)

The article also notes that dividend-paying stocks, particularly in the banking sector, continue to offer attractive yields. It specifically mentions Sansiri Pcl (SIRI) and Land and Houses Pcl (LH) as luxury real estate projects performing well and providing decent dividends. Electricity Generating Pcl (EGCO) and Ratch Group Pcl (RATCH) are mentioned for their high dividend yields and relatively low prices.

**Summary:**

Based on the crawled articles, several stocks were mentioned as potentially good investments:

*   **From Bangko